**Using KNN with both Cosine and Ecludien, you can find n numbers of next recommended movies**

Installing fuzzywuzzy which help us find movie name its not case sensitive you dont need to specify movie release year.

In [58]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import process
from tqdm import tqdm
import math
from collections import Counter
from scipy.spatial.distance import mahalanobis

In [60]:
df = pd.read_csv('/content/movies_dataset.csv')

In [61]:
df

,title,1,2,3,4,5,6,7,8,9,...,601,602,603,604,605,606,607,608,609,610
0,10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
3,28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
4,300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,X2: X-Men United (2003),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
446,You've Got Mail (1998),0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
447,Young Frankenstein (1974),5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
448,Zombieland (2009),0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5


**Checking for null values**

In [62]:
df.isnull().sum()

title    0
1        0
2        0
3        0
4        0
        ..
606      0
607      0
608      0
609      0
610      0
Length: 607, dtype: int64

**Extracting column names**

In [63]:
column_names = df.columns[1:]
column_names

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '601', '602', '603', '604', '605', '606', '607', '608', '609', '610'],
      dtype='object', length=606)

In [64]:
df.describe()

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
count,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,...,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000
mean,1.140000,0.130000,0.010000,0.648889,0.264444,0.795556,0.728889,0.344444,0.124444,0.307778,...,0.734444,0.651111,1.495556,0.402222,0.577778,1.967778,0.844444,2.297778,0.175556,2.252222
std,1.964144,0.708908,0.107668,1.476259,0.964320,1.616446,1.536435,1.104366,0.732678,1.034616,...,1.668199,1.442456,1.943805,1.146504,1.361070,1.932481,1.649128,1.751930,0.747872,2.172219
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000,0.000000,3.000000,0.000000,3.000000
75%,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.000000,0.000000,0.000000,4.000000,0.000000,4.000000,0.000000,4.500000
max,5.000000,5.000000,2.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000


#Encoding movie titles so that we have indexing for movies.

In [65]:
label_object = LabelEncoder()
movies_id = df['title']
df['title'] = label_object.fit_transform(df['title'])

In [66]:
df

,title,1,2,3,4,5,6,7,8,9,...,601,602,603,604,605,606,607,608,609,610
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,445,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
446,446,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
447,447,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
448,448,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5


#Movies_id array which stores movie_names against each id so that later when we return ID we can get movie name from here.

In [67]:
movies_id

0      10 Things I Hate About You (1999)
1                    12 Angry Men (1957)
2           2001: A Space Odyssey (1968)
3                   28 Days Later (2002)
4                             300 (2007)
                     ...                
445              X2: X-Men United (2003)
446               You've Got Mail (1998)
447            Young Frankenstein (1974)
448                    Zombieland (2009)
449                     Zoolander (2001)
Name: title, Length: 450, dtype: object

#Ratings is all users rating against each movies this will be passed to KNN.

In [68]:
ratings = df.drop('title',axis=1)
ratings = np.array(ratings)

#Finding magnitude of given movie rating

In [69]:
#Function to find magnitude for given rating. it accepts 1-D array
def magnitude(imgarray):
  magnitudevalue = 0
  for i in imgarray:
    magnitudevalue += math.pow(i,2);
  return np.sqrt(magnitudevalue)

#KNN Model with all two distances(Eucludien and Cosine)

In [70]:
class KNN():

  __test_size = 1
  __train_size = 1
  __predict_list = []
  k=0
  train_data = []
  test_data = []
  
  
  #Constructor which will load initial data
  def __init__(self,train_data,test_data,kneighbors):
     
    self.train_data = train_data
    self.__train_size = len(train_data)

    self.test_data = test_data
    self.k = (kneighbors+1)
    self.__distances_list = [[i, 0] for i in range(self.__train_size)]
    
  #Calculate distance default='euclidean' or you can pass other two while calling this function
  def CalculateDistance(self,category='euclidean'):
    
    if(category.lower()=='cosine'):
      test_img = self.test_data
      mag_test = magnitude(test_img)
      #distance_list store distance between test rating and each training rating
      for i in range(len(self.train_data)):
        dotproduct = np.dot(self.train_data[i],test_img)
        self.__distances_list[i][1] = 1-(dotproduct/(mag_test*(magnitude(self.train_data[i]))))

      #sorted list contains sorted values of distance_list in ascending order   
      sorted_lst = sorted(self.__distances_list, key=lambda x: x[1])
      
      movies_indexes = [item[0] for item in sorted_lst[:self.k]]
      movies_distances = [item[1] for item in sorted_lst[:self.k]]
      #storing prediction of KNN for given movie rating
      return movies_indexes[1:],movies_distances[1:]

    else:

      for i in range(len(self.train_data)):
        
        squared_diff = np.square(self.test_data - self.train_data[i])
        sum_squared_diff = np.sum(squared_diff)
        distance = np.sqrt(sum_squared_diff)
        
        self.__distances_list[i][1] = distance
      
        #sorted list contains sorted values of distance_list in ascending order   
      sorted_lst = sorted(self.__distances_list, key=lambda x: x[1])
      
      movies_indexes = [item[0] for item in sorted_lst[:self.k]]
      movies_distances = [item[1] for item in sorted_lst[:self.k]]
      #storing prediction of KNN for given movie rating
      return movies_indexes[1:],movies_distances[1:]

#Recommendation Function requires three parameters inside function you can set KNN to find recommendation based on Euclidean or Cosine distance.
movie_name = Given movie for which you find next same recommendation

data = It requires data from which you want recommendations

neighbours = It required how many movies you want in recommendations

In [71]:
def Recommendation(movie_name,data,neighbours):
  
  index = process.extractOne(movie_name,movies_id)[2]
  print('Movie Watched: ',movies_id[index],' Index: ',index)
  knn_obj = KNN(data,data[index],neighbours)
  recommended_index,recommended_distance = knn_obj.CalculateDistance('euclidean')
  print(f"\nTop {neighbours} recommended movies are: \n")
  for i,index in enumerate(recommended_index):
    print("Distance: {:20}    Movie: {}".format(recommended_distance[i], movies_id[index]))

#Calling Recommendation Functions

In [72]:
required_recommendations = 1
Recommendation('Splash',ratings,required_recommendations)

Movie Watched:  Splash (1984)  Index:  375

Top 1 recommended movies are: 

Distance:    23.71181140275875    Movie: Honey, I Shrunk the Kids (1989)
